In [2]:
from xmitgcm import open_mdsdataset
from plotters import LLC_plotter

# Set as appropriate; I've saved only the grid data files in a dedicated
# directory.
data_dir = '/home/sean/Desktop/test-data'
grid_dir = '/home/sean/Desktop/grid-data'
ds = open_mdsdataset(grid_dir=grid_dir, data_dir=data_dir, delta_t=3600, geometry='llc')

# Make a plotter with the dataset as the parent. Note that if you modify the data set
# stored in `ds`, changes *are* stored in the plotter's `parent` variable. However, if
# you rebind `ds`, the plotter still sees the last state of the data set.
plotter = LLC_plotter(ds)

# Have a look at the data set that the plotter stores internally:
plotter.ds

/home/sean/miniconda3/lib/python3.6/site-packages/xmitgcm/mds_store.py:696: UserWarning: Couldn't find available_diagnostics.log in /home/sean/Desktop/test-data. Using default version.
  "in %s. Using default version." % data_dir)
/home/sean/miniconda3/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home/sean/miniconda3/lib/python3.6/site-packages/xmitgcm/mds_store.py:220: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


<xarray.Dataset>
Dimensions:  (i: 90, i_g: 90, j: 1170, j_g: 1170)
Coordinates:
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
    XC       (j, i) float32 dask.array<shape=(1170, 90), chunksize=(90, 90)>
    XG       (j_g, i_g) float32 dask.array<shape=(1170, 90), chunksize=(90, 90)>
    YC       (j, i) float32 dask.array<shape=(1170, 90), chunksize=(90, 90)>
    YG       (j_g, i_g) float32 dask.array<shape=(1170, 90), chunksize=(90, 90)>
Data variables:
    *empty*

In [3]:
# Now compute some data variable that we want to plot, and
# store it in the data set.
ds['THETA_vol_avg'] = (ds.THETA * ds.rA * ds.drF).sum(dim='k')
plotter.set_data_variable('THETA_vol_avg')

# And now the internal data set looks like:
plotter.ds

<xarray.Dataset>
Dimensions:      (i: 90, i_g: 90, j: 1170, j_g: 1170, time: 6)
Coordinates:
  * j            (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * i            (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * j_g          (j_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
  * i_g          (i_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
    XC           (j, i) float32 dask.array<shape=(1170, 90), chunksize=(90, 90)>
    XG           (j_g, i_g) float32 dask.array<shape=(1170, 90), chunksize=(90, 90)>
    YC           (j, i) float32 dask.array<shape=(1170, 90), chunksize=(90, 90)>
    YG           (j_g, i_g) float32 dask.array<shape=(1170, 90), chunksize=(90, 90)>
  * time         (time) int64 2635200 5140800 7819200 10411200 13089600 15681600
Data variables:
    var_to_plot  (time, j, i) float32 dask.array<shape=(6, 1170, 90), chunksize=(6, 90, 90)>

Actual plotting coming soon; I haven't got GeoViews examples to work.